In [ ]:
from platform import python_version
print(python_version())

In [ ]:
#https://crime-data-explorer.app.cloud.gov/explorer/state/georgia/crime

In [2]:
import pandas as pd
import numpy as np, sys, time, pdb, matplotlib.pyplot as plt
from pymongo import MongoClient

In [ ]:
pd.set_option('display.max_rows', 100)# None

In [ ]:
data_2020 = pd.read_csv('./flask/static/data/COBRA-2020.csv')
data_2021 = pd.read_csv('./flask/static/data/COBRA-2021.csv')
data_2009_2019 = pd.read_csv('./flask/static/data/COBRA-2009-2019.csv', low_memory=False)

In [ ]:
data_2009_2019.describe()

In [ ]:
data_2020_list = data_2020.columns.values.tolist()
data_2021_list = data_2021.columns.values.tolist()
data_2009_2019_list = data_2009_2019.columns.values.tolist()

In [ ]:
print(len(data_2020_list))
data_2020.head(1)

In [ ]:
print(len(data_2009_2019_list))
data_2009_2019.tail(10)

In [ ]:
#Since the 2009-2019 dataset is the largest, everything will try and have the same columns as that dataset
data_2020.rename(columns = {
 'offense_id' : 'Report Number',
 'rpt_date' : 'Report Date',
 'occur_date' : 'Occur Date',
 'occur_time' : 'Occur Time',
 'poss_date' : 'Possible Date',
 'poss_time' : 'Possible Time',
 'beat' : 'Beat',
 'apt_office_prefix' : 'Apartment Office Prefix',
 'apt_office_num' : 'Apartment Number',
 'location' : 'Location',
 #'MinOfucr' : ,
 #'dispo_code' : ,
 'Shift' : 'Shift Occurence',
 'loc_type' : 'Location Type',
 'UC2_Literal': 'UCR Literal',
 'ibr_code' : 'IBR Code',
 'neighborhood' : 'Neighborhood',
 'npu' : 'NPU',
 'long' : 'Longitude',
 'lat' : 'Latitude'
}, inplace = True)
#Drop columns that do not seem to show up in 2009-2019 dataset
data_2020 = data_2020.drop(columns=['MinOfucr','dispo_code'])
data_2020.head(1)

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)

In [ ]:
#It can be seen that the 'UCR #' shows up only in the 2009-2019 data and realistically serves no purpose because its a code only a cop would udnerstand
#So it should be droped to simplify
data_2009_2019 = data_2009_2019.drop(columns=['UCR #'])

In [ ]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)
#There should now be no differences as at all

In [ ]:
#Concat the dataframes together
data = pd.concat([data_2009_2019, data_2020])

In [ ]:
data_2009_2019['Occur Date'].sort_values()

In [ ]:
data['Report Date'] = pd.to_datetime(data['Report Date'])
#data['Occur Date'] = pd.to_datetime(data['Occur Date'])

In [ ]:
#data = data.astype({'Report Date':'datetime64','Occur Date':'datetime64','Possible Date':'datetime64'},errors = 'raise')

In [ ]:
data.dtypes

In [ ]:
data.describe

In [ ]:
data.to_csv('data.csv')

In [ ]:
df.reset_index(drop=True, inplace=True)
df.to_json('data.json')

# Machine Learnings

## Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df[["Latitude","Longitude","Neighborhood"]]
df = df.dropna()

## Create a Train Test Split

Use `Neighborhood` for the y values
This can be an idiot check to see if lat an lon can predict neighborhood

In [ ]:
from sklearn.model_selection import train_test_split
y = df["Neighborhood"]
X = df.drop(columns=["Neighborhood"])

#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
y_train.nunique()

# Train the Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)# Fit the LR model

In [ ]:
df

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
model.predict()

In [ ]:
X_scaler.transform(X_train)[0]

In [ ]:
model.predict(X_train_scaled)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [1, 3,5,7,9],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

In [ ]:
idk = grid.fit(X_train_scaled, y_train)
idk

# Need to Decrease Sample Size

It has become apparent that there is too mach data and our relatively weak computers are not up to the challenge give the time.

It was decided to just look at 5 major "Neighborhood" of interest:

buckhead, midtown, brookhaven, downtown, old fourth ward

In [ ]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df_reduced = df[["Latitude","Longitude","Neighborhood"]]
#df_reduced = df_reduced.dropna()
df_reduced = df

In [ ]:
df_reduced["Neighborhood"].unique();

In [ ]:
#Used to show all rows for error checking
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 100)

In [ ]:
df_buckhead = df_reduced[(df_reduced["Neighborhood"] == "Buckhead Forest") |(df_reduced["Neighborhood"] == "Buckhead Village")
           |(df_reduced["Neighborhood"] == "North Buckhead") |(df_reduced["Neighborhood"] == "Buckhead Heights")
          ]
df_buckhead

In [ ]:
df_buckhead['Neighborhood']

In [ ]:
df_buckhead = df_buckhead.assign(Neighborhood = 'Buckhead' )
df_buckhead

In [ ]:
df_other_hoods  = df_reduced[(df_reduced["Neighborhood"] == "Midtown") | (df_reduced["Neighborhood"] == "Brookhaven")
          |(df_reduced["Neighborhood"] == "Downtown") | (df_reduced["Neighborhood"] == "Old Fourth Ward")]
df_other_hoods

In [ ]:
df_5hoods = pd.concat([df_buckhead,df_other_hoods], ignore_index=True, sort=False)
df_5hoods

In [ ]:
df_5hoods = df_5hoods.drop(columns=['Unnamed: 0'])
df_5hoods.to_csv('data_5hoods.csv', index= False)

# Machine Learnings

In [73]:
df = pd.read_csv("data_5hoods.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [74]:
df = df.drop(columns=['Apartment Office Prefix',
                      'Report Date',
                      'Occur Date',
                      'Occur Time',
                      'Possible Time',
                      'Apartment Number',
                      'Location',
                      'Shift Occurence',
                      'NPU',
                      'Possible Date',
                      'IBR Code',
                      'Location Type',
                     ])
df.head(1)

,Report Number,Beat,UCR Literal,Neighborhood,Latitude,Longitude
0,90010745,208.0,LARCENY-FROM VEHICLE,Buckhead,33.85115,-84.37944


In [75]:
pd.set_option('display.max_rows', 1000)# None
df = df.dropna()
df[pd.to_numeric(df['Beat'], errors='coerce').notnull()]
#df[pd.to_numeric(df['Location Type'], errors='coerce').notnull()]
print(df.dtypes)
df.head(1)

Report Number      int64
Beat             float64
UCR Literal       object
Neighborhood      object
Latitude         float64
Longitude        float64
dtype: object


,Report Number,Beat,UCR Literal,Neighborhood,Latitude,Longitude
0,90010745,208.0,LARCENY-FROM VEHICLE,Buckhead,33.85115,-84.37944


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
y = df["UCR Literal"]
X = df.drop(columns=["UCR Literal"])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 
X['Neighborhood']= le.fit_transform(X['Neighborhood']) 

In [77]:
X

,Report Number,Beat,Neighborhood,Latitude,Longitude
0,90010745,208.0,1,33.851150,-84.379440
1,90010159,206.0,1,33.837520,-84.375740
2,90021591,208.0,1,33.848830,-84.381740
3,90020096,208.0,1,33.847090,-84.372950
4,90020730,208.0,1,33.847120,-84.373850
...,...,...,...,...,...
65538,202741548,502.0,3,33.783060,-84.379421
65539,202740394,508.0,2,33.763927,-84.396356
65540,202740612,603.0,4,33.768814,-84.366527
65541,202740728,503.0,3,33.785508,-84.383434


In [78]:
# enc = OneHotEncoder(handle_unknown='ignore')
# enc.fit(X)
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [79]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [6,7,11],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [6, 7, 11], 'penalty': ['l1', 'l2', 'none']},
             verbose=3)

In [81]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ................................C=6, penalty=l1; total time=  12.0s
[CV 2/5] END ................................C=6, penalty=l1; total time=  11.3s
[CV 3/5] END ................................C=6, penalty=l1; total time=  11.5s


/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 4/5] END ................................C=6, penalty=l1; total time=  11.0s
[CV 5/5] END ................................C=6, penalty=l1; total time=  11.3s
[CV 1/5] END ................................C=6, penalty=l2; total time=   0.5s
[CV 2/5] END ................................C=6, penalty=l2; total time=   0.5s
[CV 3/5] END ................................C=6, penalty=l2; total time=   0.4s
[CV 4/5] END ................................C=6, penalty=l2; total time=   0.4s
[CV 5/5] END ................................C=6, penalty=l2; total time=   0.4s
[CV 1/5] END ..............................C=6, penalty=none; total time=   0.0s
[CV 2/5] END ..............................C=6, penalty=none; total time=   0.0s
[CV 3/5] END ..............................C=6, penalty=none; total time=   0.0s
[CV 4/5] END ..............................C=6, penalty=none; total time=   0.0s
[CV 5/5] END ..............................C=6, penalty=none; total time=   0.0s


/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

[CV 1/5] END ................................C=7, penalty=l1; total time=  12.6s
[CV 2/5] END ................................C=7, penalty=l1; total time=  13.0s
[CV 3/5] END ................................C=7, penalty=l1; total time=  12.2s


...

[CV 4/5] END ...............................C=11, penalty=l1; total time=  13.3s


/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV 5/5] END ...............................C=11, penalty=l1; total time=  14.3s
[CV 1/5] END ...............................C=11, penalty=l2; total time=   0.5s
[CV 2/5] END ...............................C=11, penalty=l2; total time=   0.5s
[CV 3/5] END ...............................C=11, penalty=l2; total time=   0.5s
[CV 4/5] END ...............................C=11, penalty=l2; total time=   0.5s
[CV 5/5] END ...............................C=11, penalty=l2; total time=   0.5s
[CV 1/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 2/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 3/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 4/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 5/5] END .............................C=11, penalty=none; total time=   0.0s


/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [6, 7, 11], 'penalty': ['l1', 'l2', 'none']},
             verbose=3)

In [82]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 6, 'penalty': 'l1'}
0.45191193144366615


# Save the Model

In [ ]:
# save fitted model to file
# In the specific case of scikit-learn, it may be better to use joblib’s replacement of
# pickle (dump & load), which is more efficient on objects that carry large numpy arrays
# internally as is often the case for fitted scikit-learn estimators, but can only pickle
# to the disk and not to a string:

import joblib
filename = 'five_hoods_LR.sav'
joblib.dump(grid, filename)

## Test

In [ ]:
Model = joblib.load("five_hoods_LR.sav")
Model.predict(X_test_scaled)